# TLT SSD example usecase

本教程将详细讲述如何利用NVIDIA Transfer Learning toolkit从头开始训练一个口罩检测的模型，并将模型转换成可以直接部署在NVIDIA GPU（Tesla & Jetson）上的格式.

0. [设置环境变量](#head-0)
1. [准备数据集和预训练模型](#head-1) <br>
    1.1 [将数据集转换成KITTI格式并生成TFrecord文件](#head-1-1) <br>
    1.2 [下载预训练模型](#head-1-2) <br>
2. [设置训练参数](#head-2)
3. [利用NVIDIA Transfer Learning Toolkit训练模型](#head-3)
4. [评估模型](#head-4)
5. [模型剪枝](#head-5)
6. [重新训练剪枝后的模型](#head-6)
7. [评估重新训练的模型](#head-7)
8. [可视化推理过程](#head-8)
9. [模型的导出和部署](#head-9)
10. [确认导出模型](#head-10)

## 0. 设置环境变量 <a class="anchor" id="head-0"></a>


In [1]:
# 设置环境变量，这里请注意KEY是需要您从NGC官网申请的，您可以从以下网址得到您自己的KEY：
#https://ngc.nvidia.com/catalog
#USER_EXPERIMENT_DIR是我们实验的目录
#DATA_DOWNLOAD_DIR是用来保存我们的数据样本和预训练模型
#SPECS_DIR是用来保存我们训练设置的超参
#以下设置仅代表作者本人的设置目录，需要用户根据自己的实际情况设置更新
print("Please replace the variable with your key.")
%set_env KEY=OHB1YTZ0Z2RxYTBzdnE3YTNpcnVydmM4cXI6OGVkNDU4ZGQtNjViOC00NzYxLWFhMDUtMjgxMDQ2ZTVmNzAx
%set_env USER_EXPERIMENT_DIR=/workspace/tlt_docker_files/mydata/tlt-tensorrt-nano
%set_env DATA_DOWNLOAD_DIR=/workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data
%set_env SPECS_DIR=/workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/specs
!mkdir -p $USER_EXPERIMENT_DIR
!mkdir -p $DATA_DOWNLOAD_DIR
!mkdir -p $SPECS_DIR

Please replace the variable with your key.
env: KEY=OHB1YTZ0Z2RxYTBzdnE3YTNpcnVydmM4cXI6OGVkNDU4ZGQtNjViOC00NzYxLWFhMDUtMjgxMDQ2ZTVmNzAx
env: USER_EXPERIMENT_DIR=/workspace/tlt_docker_files/mydata/tlt-tensorrt-nano
env: DATA_DOWNLOAD_DIR=/workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data
env: SPECS_DIR=/workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/specs


## 1. 准备训练数据集和下载预训练模型 <a class="anchor" id="head-1"></a>

In [2]:
#验证KEY是否设置成功，这个KEY非常重要，用户训练出来的模型在导出和转换时都需要
!echo $KEY

OHB1YTZ0Z2RxYTBzdnE3YTNpcnVydmM4cXI6OGVkNDU4ZGQtNjViOC00NzYxLWFhMDUtMjgxMDQ2ZTVmNzAx


 NVIDIA官方教程中使用的KITTI目标检测数据集，您可以通过以下网址下载：
 http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d.
 
 图片下载网址： (http://www.cvlibs.net/download.php?file=data_object_image_2.zip) 
 
 标注下载网址： (http://www.cvlibs.net/download.php?file=data_object_label_2.zip) 
 
 下载好之后，把数据集放到$DATA_DOWNLOAD_DIR文件夹中.

In [3]:
# 查看上面下载的数据----这步当前实验可以不运行
!mkdir -p $DATA_DOWNLOAD_DIR
!if [ ! -f $DATA_DOWNLOAD_DIR/data_object_image_2.zip ]; then echo 'Image zip file not found, please download.'; else echo 'Found Image zip file.';fi
!if [ ! -f $DATA_DOWNLOAD_DIR/data_object_label_2.zip ]; then echo 'Label zip file not found, please download.'; else echo 'Found Labels zip file.';fi

Found Image zip file.
Found Labels zip file.


In [3]:
# 解压----这步当前实验可以不运行
# !unzip -u $DATA_DOWNLOAD_DIR/data_object_image_2.zip -d $DATA_DOWNLOAD_DIR
# !unzip -u $DATA_DOWNLOAD_DIR/data_object_label_2.zip -d $DATA_DOWNLOAD_DIR

Archive:  /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/data_object_image_2.zip
   creating: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/002480.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/005952.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/000709.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/000814.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/006192.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/006017.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/002731.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/005295.png  
 extracting

 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/004036.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/006216.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/001394.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/000894.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/003293.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/002474.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/006360.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/007271.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/004064.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/001

 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/001020.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/005582.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/000965.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/000313.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/002161.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/000969.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/002599.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/006832.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/003783.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/002

 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/002012.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/000734.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/006507.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/002726.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/001339.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/004212.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/007037.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/003928.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/002506.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/001

 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/003826.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/000740.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/002402.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/006029.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/004177.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/005802.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/005150.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/004807.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/001940.png  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/image_2/002

...

 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/003183.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/004981.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/007413.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/004342.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/002365.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/004835.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/005243.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/004095.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/006303.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/007

 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/003875.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/006909.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/003595.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/002299.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/004725.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/002998.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/005304.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/004867.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/002930.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/005

 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/003137.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/004570.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/003473.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/006779.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/003114.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/004765.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/005868.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/000419.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/006354.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/002

 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/006880.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/005952.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/006836.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/004987.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/005301.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/000558.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/002808.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/002053.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/006044.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/005

 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/001992.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/000469.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/000688.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/006731.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/002477.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/004375.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/001326.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/000313.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/000569.txt  
 extracting: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/label_2/004

In [4]:
# 确认
!ls -l $DATA_DOWNLOAD_DIR/

total 12280824
-rwxrwxrwx 1 1000 1000 12569945557 Jun 28 05:00 data_object_image_2.zip
-rw-r--r-- 1 root root     5601213 May 11  2018 data_object_label_2.zip
drwxr-xr-x 3 root root        4096 Jun 28 09:15 testing
drwxr-xr-x 4 root root        4096 Jun 28 09:16 training


* 我们本次实验使用的是网上多个数据集混合的训练样本，VOC格式的.xml数据标注方式，需要用户自己讲标注文件转换成KITTI的格式

* 在本次实验当前的目录下，‘data’文件夹下的‘images’保存的是处理好的训练图片，‘labels’文件夹保存的是训练样本的标注文件

* "xmlfiles"文件夹保存的是转换前的标注文件

* "xml2kitti"文件夹保存的是转换VOC的.xml文件到KITT格式的工具

* changename.py是用来统一更改后缀名的工具

* check.py是用来检查图片是否有损坏的工具（比如某个作者之前遇到的错误是某个图片以‘BM’开头，系统会报告格式错误）

### 1.1 将准备好的训练数据从KITTI格式生成TFrecords格式 <a class="anchor" id="head-1-1"></a>

* 更改在specs文件夹中的ssd_tfrecords_kitti_trainval.txt中的参数，指定训练样本的路径
* 利用tlt-dataset-convert 生成TFrecords文件
* TFRecords文件只需要生成一次既可
* 注意这里的路径都是作者本人的环境路径，请用户自行更换成自己的环境路径

In [5]:
print("TFrecords conversion spec file for training")
!cat $SPECS_DIR/ssd_tfrecords_kitti_trainval.txt

TFrecords conversion spec file for training
kitti_config {
  root_directory_path: "/workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training"
  image_dir_name: "images"
  label_dir_name: "labels"
  image_extension: ".png"
  partition_mode: "random"
  num_partitions: 2
  val_split: 14
  num_shards: 10
}
image_directory_path: "/workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/data/training/images"

In [6]:
# 创建一个新的文件夹来存储tfrecords文件.
!mkdir -p $USER_EXPERIMENT_DIR/tfrecords
#创建tfrecords文件 ‘-d’指的是设置文件，就是我们上一步更改的那个，‘-o’指的是输出
#目录。
!tlt-dataset-convert -d $SPECS_DIR/ssd_tfrecords_kitti_trainval.txt \
                     -o $USER_EXPERIMENT_DIR/tfrecords/kitti_trainval/kitti_trainval

Using TensorFlow backend.
2020-06-28 11:01:57,450 - iva.detectnet_v2.dataio.build_converter - INFO - Instantiating a kitti converter
2020-06-28 11:01:57,451 - iva.detectnet_v2.dataio.dataset_converter_lib - INFO - Creating output directory /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/tfrecords/kitti_trainval
2020-06-28 11:01:57,490 - iva.detectnet_v2.dataio.kitti_converter_lib - INFO - Num images in
Train: 6434	Val: 1047
2020-06-28 11:01:57,490 - iva.detectnet_v2.dataio.kitti_converter_lib - INFO - Validation data in partition 0. Hence, while choosing the validationset during training choose validation_fold 0.
2020-06-28 11:01:57,494 - iva.detectnet_v2.dataio.dataset_converter_lib - INFO - Writing partition 0, shard 0
/usr/local/lib/python2.7/dist-packages/iva/detectnet_v2/dataio/kitti_converter_lib.py:266: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
2020-06-28 11:0

In [7]:
#查看生成的文件
!ls -rlt $USER_EXPERIMENT_DIR/tfrecords/kitti_trainval

total 7136
-rw-r--r-- 1 root root 104459 Jun 28 11:01 kitti_trainval-fold-000-of-002-shard-00000-of-00010
-rw-r--r-- 1 root root 102600 Jun 28 11:02 kitti_trainval-fold-000-of-002-shard-00001-of-00010
-rw-r--r-- 1 root root 102049 Jun 28 11:02 kitti_trainval-fold-000-of-002-shard-00002-of-00010
-rw-r--r-- 1 root root  97532 Jun 28 11:02 kitti_trainval-fold-000-of-002-shard-00003-of-00010
-rw-r--r-- 1 root root  98063 Jun 28 11:02 kitti_trainval-fold-000-of-002-shard-00004-of-00010
-rw-r--r-- 1 root root 103583 Jun 28 11:02 kitti_trainval-fold-000-of-002-shard-00005-of-00010
-rw-r--r-- 1 root root 101843 Jun 28 11:02 kitti_trainval-fold-000-of-002-shard-00006-of-00010
-rw-r--r-- 1 root root 100453 Jun 28 11:02 kitti_trainval-fold-000-of-002-shard-00007-of-00010
-rw-r--r-- 1 root root  99149 Jun 28 11:02 kitti_trainval-fold-000-of-002-shard-00008-of-00010
-rw-r--r-- 1 root root 109910 Jun 28 11:02 kitti_trainval-fold-000-of-002-shard-00009-of-00010
-rw-r--r-- 1 root root 625983 Jun 28 11

### 1.2 下载预训练模型 <a class="anchor" id="head-1-2"></a>

您可以通过一下命令下载模型，也可以通过手动的方式下载预训练模型：
[ngc.nvidia.com](ngc.nvidia.com)

In [8]:
#查看当前目标检测可用的预训练模型
!ngc registry model list nvidia/tlt_pretrained_object_detection:*

+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| Versi | Accur | Epoch | Batch | GPU   | Memor | File  | Statu | Creat |
| on    | acy   | s     | Size  | Model | y Foo | Size  | s     | ed    |
|       |       |       |       |       | tprin |       |       | Date  |
|       |       |       |       |       | t     |       |       |       |
+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| resne | 74.38 | 80    | 1     | V100  | 38.3  | 38.31 | UPLOA | Apr   |
| t10   |       |       |       |       |       | MB    | D_COM | 29,   |
|       |       |       |       |       |       |       | PLETE | 2020  |
| resne | 76.74 | 80    | 1     | V100  | 89.0  | 88.96 | UPLOA | Apr   |
| t18   |       |       |       |       |       | MB    | D_COM | 29,   |
|       |       |       |       |       |       |       | PLETE | 2020  |
| resne | 77.04 | 80    | 1     | V100  | 170.7 | 170.6 | UPLOA | Apr   |
| t34   |       |       |       |     

In [9]:
#创建一个新的文件夹，存储下载好的预训练模型
!mkdir -p $USER_EXPERIMENT_DIR/pretrained_mobilenet_v2/tlt_pretrained_object_detection_mobilenet_v2

In [10]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tlt_pretrained_object_detection:mobilenet_v2 --dest $USER_EXPERIMENT_DIR/pretrained_mobilenet_v2/tlt_pretrained_object_detection_mobilenet_v2

Downloaded 4.3 MB in 14s, Download speed: 313.95 KB/s                
----------------------------------------------------
Transfer id: tlt_pretrained_object_detection_vmobilenet_v2 Download status: Completed.
Downloaded local path: /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/pretrained_mobilenet_v2/tlt_pretrained_object_detection_mobilenet_v2/tlt_pretrained_object_detection_vmobilenet_v2
Total files downloaded: 1 
Total downloaded size: 4.3 MB
Started at: 2020-06-28 09:22:01.245271
Completed at: 2020-06-28 09:22:15.266408
Duration taken: 14s
----------------------------------------------------


In [8]:
print("查看下载好的预训练模型.")
!ls -l $USER_EXPERIMENT_DIR/pretrained_mobilenet_v2/tlt_pretrained_object_detection_mobilenet_v2

查看下载好的预训练模型.
total 4
drwx------ 2 root root 4096 Jun 28 09:22 tlt_pretrained_object_detection_vmobilenet_v2


## 2. 设置训练的超参 <a class="anchor" id="head-2"></a>
* 训练数据集
    * 为了使用最新生成TFrecords文件，在 `$SPECS_DIR/ssd_train_resnet18_kitti.txt`设置文件路径 
    * Update the fold number to use for evaluation. In case of random data split, please use fold 0 only
    * 更新测试样本数量，如果随机生成，可以设置成0
* 预训练模型
* 其他超参比如batch size，learning rate等等

In [10]:
!cat $SPECS_DIR/ssd_train_mobilenet_v2_kitti.txt

random_seed: 42
ssd_config {
  aspect_ratios_global: "[1.0, 2.0, 0.5, 3.0, 1.0/3.0]"
  scales: "[0.05, 0.1, 0.25, 0.4, 0.55, 0.7, 0.85]"
  two_boxes_for_ar1: true
  clip_boxes: false
  loss_loc_weight: 0.8
  focal_loss_alpha: 0.25
  focal_loss_gamma: 2.0
  variances: "[0.1, 0.1, 0.2, 0.2]"
  arch: "mobilenet_v2"
  freeze_bn: false
}
training_config {
  batch_size_per_gpu: 64
  num_epochs: 100
  learning_rate {
  soft_start_annealing_schedule {
    min_learning_rate: 5e-7
    max_learning_rate: 5e-3
    soft_start: 0.1
    annealing: 0.3
    }
  }
  regularizer {
    type: L2
    weight: 0.00004
  }
}
eval_config {
  validation_period_during_training: 5
  average_precision_mode: SAMPLE
  batch_size: 32
  matching_iou_threshold: 0.5
}
nms_config {
  confidence_threshold: 0.01
  clustering_iou_threshold: 0.6
  top_k: 200
}
augmentation_config {
  preprocessing {
    output_image_width: 300
    output_image_height: 300
    output_image_channel: 3
    crop_right: 300
    crop_bottom: 300
  

## 3. 开始训练模型 <a class="anchor" id="head-3"></a>
* 需要提供预训练模型、specs文件夹中的设置文件地址以及输出文件夹
* 注意：训练课程会持续好几个小时或者一天

In [11]:
!mkdir -p $USER_EXPERIMENT_DIR/experiment_dir_unpruned

In [12]:
print("如果想用多个GPU可以更改 ‘--gpus’的参数")
!tlt-train ssd -e $SPECS_DIR/ssd_train_mobilenet_v2_kitti.txt \
               -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned \
               -k $KEY \
               -m /workspace-hekun/mydata/tlt-tensorrt-nano/pretrained_mobilenet_v2/tlt_pretrained_object_detection_mobilenet_v2/mobilenet_v2.hdf5 \
               --gpus 1

如果想用多个GPU可以更改 ‘--gpus’的参数
Using TensorFlow backend.
2020-06-28 11:22:16,382 [INFO] /usr/local/lib/python2.7/dist-packages/iva/ssd/utils/spec_loader.pyc: Merging specification from /workspace/tlt_docker_files/mydata/tlt-tensorrt-nano/specs/ssd_train_mobilenet_v2_kitti.txt
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match 

target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is not updated to match the crop areaif the dataset contains target/truncation.
target/truncation is

In [ ]:
#查看训练出来的模型
!ls /workspace-hekun/mydata/tlt-tensorrt-nano/experiment_dir_unpruned/weights/

In [ ]:
print("To resume from checkpoint, please uncomment and run this instead. Change last two arguments accordingly.")
# !tlt-train ssd -e $SPECS_DIR/ssd_train_resnet18_kitti.txt \
#                -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned \
#                -k $KEY \
#                -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/ssd_resnet18_epoch_001.tlt \
#                --gpus 1 \
#                --initial_epoch 2 

In [ ]:
#查看训练出来的模型
print('Model for each epoch:')
print('---------------------')
!ls -ltrh $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights

In [ ]:
# 接下来通过评估的测试结果来选择一个训练效果最好的模型
# Note csv epoch number is 1 less than model file epoch. For example, epoch 79 in csv corresponds to _080.tlt
!cat $USER_EXPERIMENT_DIR/experiment_dir_unpruned/ssd_training_log_mobilenet_v2.csv
%set_env EPOCH=100

## 4. 评估训练好的模型 <a class="anchor" id="head-4"></a>

In [ ]:
!tlt-evaluate ssd -e $SPECS_DIR/ssd_train_mobilenet_v2_kitti.txt \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/ssd_mobilenet_v2_epoch_$EPOCH.tlt \
                  -k $KEY

## 5. 模型剪枝 <a class="anchor" id="head-5"></a>
* 设置剪枝的阈值
* 设置选择剪枝的模型，也就是我们之前通过评估选择的那个模型
* 设置输出路径

通常，我们需要设置‘-pth’参数来平衡模型的精度与模型的大小（或者说速度）。更高的‘-pth’数值，会让模型更小（更快的推理速度），但是也会降低模型精度。在本次实验中，作者使用的是0.5，如果精度没问题，我们可以增加‘-pth’的数值，来进一步剪枝。反之，我们则需要减小‘-pth’的数值。

In [ ]:
#创建新的文件夹，保存剪枝后的模型
!mkdir -p $USER_EXPERIMENT_DIR/experiment_dir_pruned

In [ ]:
#开始模型剪枝
!tlt-prune -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/ssd_mobilenet_v2_epoch_$EPOCH.tlt \
           -o $USER_EXPERIMENT_DIR/experiment_dir_pruned/ssd_mobilenet_v2_pruned.tlt \
           -eq intersection \
           -pth 0.5 \
           -k $KEY

In [ ]:
#查看剪枝后的模型
!ls -rlt $USER_EXPERIMENT_DIR/experiment_dir_pruned/

## 6. 重新训练剪枝后的模型 <a class="anchor" id="head-6"></a>
* 模型在剪枝后需要重新训练
* 还需要定义重新训练的参数，比如学习率等。
* 注意：重新训练也需要比较长的时间

In [ ]:
# 在这里我们打印出训练的超参
# 在ssd_retrain_mobilenet_v2_kitti.txt文件中我们需要定义训练设置，
# 还可以通过调整数据集来取得更好的训练效果
!cat $SPECS_DIR/ssd_retrain_mobilenet_v2_kitti.txt

In [ ]:
# 创建一个新的文件夹，来保存重新训练的模型
!mkdir -p $USER_EXPERIMENT_DIR/experiment_dir_retrain

In [ ]:
# 使剪之后的模型作为预训练模型，重新训练
!tlt-train ssd --gpus 1 \
               -e $SPECS_DIR/ssd_retrain_mobilenet_v2_kitti.txt \
               -r $USER_EXPERIMENT_DIR/experiment_dir_retrain \
               -m $USER_EXPERIMENT_DIR/experiment_dir_pruned/ssd_mobilenet_v2_pruned.tlt \
               -k $KEY

In [ ]:
# 查看最新训练出来的模型.
!ls -rlt $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights

In [ ]:
# 现在，您可以在.CVS文件中查看模型的评估结果
# 注意.CVS文件中的序号比实际训练的序号小1, epoch 79 模型指的是第80个.tlt
!cat $USER_EXPERIMENT_DIR/experiment_dir_retrain/ssd_training_log_mobilenet_v2.csv
%set_env EPOCH=040

## 7. 评估预训练模型 <a class="anchor" id="head-7"></a>

In [ ]:
!tlt-evaluate ssd -e $SPECS_DIR/ssd_retrain_mobilenet_v2_kitti.txt \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/ssd_mobilenet_v2_epoch_$EPOCH.tlt \
                  -k $KEY

## 8. 可视化推理结果 <a class="anchor" id="head-8"></a>
在这个部分，我们利用tlt-infer在我们训练的模型的基础上进行推理，并且可视化结果

In [ ]:
# Running inference for detection on n images
!tlt-infer ssd -i $DATA_DOWNLOAD_DIR/infer_images \
               -o $USER_EXPERIMENT_DIR/data/infered_images \
               -e $SPECS_DIR/ssd_train_mobilenet_v2_kitti.txt \
               -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/ssd_mobilenet_v2_epoch_$EPOCH.tlt \
               -l $USER_EXPERIMENT_DIR/ssd_infer_labels \
               -k $KEY

`tlt-infer`工具会输出两个文件
1. 推理检测出来的图片，保存在`$USER_EXPERIMENT_DIR/ssd_infer_images`文件夹中
2. 每一帧的bounding box标注文件会以KITTI的格式保存在`$USER_EXPERIMENT_DIR/ssd_infer_labels`文件夹中

In [ ]:
# 简单的写一个可视化工具
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['USER_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx / num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# 在这里执行可视化工具
OUTPUT_PATH = "data/infered_images" # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 9. 模型的导出和部署! <a class="anchor" id="head-9"></a>

In [ ]:
!mkdir -p $USER_EXPERIMENT_DIR/export
# 这里导出的是FP32数据类型的模型，您可以通过更改--data_type的参数来更改导出的模型的数据精度
# 比如您可以设置--data_type fp16
!tlt-export ssd -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/ssd_mobilenet_v2_epoch_$EPOCH.tlt \
                -k $KEY \
                -o $USER_EXPERIMENT_DIR/export/ssd_mobilenet_v2_epoch_040.etlt \
                -e $SPECS_DIR/ssd_retrain_mobilenet_v2_kitti.txt  \
                --batch_size 1 \
                --data_type fp32

# 这里有直接导出INT 8 的数据类型的模型的命令 \
# !tlt-export ssd -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/ssd_resnet18_epoch_$EPOCH.tlt  \
#                 -o $USER_EXPERIMENT_DIR/export/ssd_resnet18_epoch_$EPOCH.etlt \
#                 -e $SPECS_DIR/ssd_retrain_resnet18_kitti.txt \
#                 -k $KEY \
#                 --cal_image_dir  $USER_EXPERIMENT_DIR/data/testing/image_2 \
#                 --data_type int8 \
#                 --batch_size 1 \
#                 --batches 10 \
#                 --cal_cache_file $USER_EXPERIMENT_DIR/export/cal.bin  \
#                 --cal_data_file $USER_EXPERIMENT_DIR/export/cal.tensorfile

In [ ]:
# 这里展示了tlt-export所有的参数
!tlt-export

In [ ]:
# 这里查看导出的模型
print('导出模型:')
print('------------')
!ls -lh $USER_EXPERIMENT_DIR/export

您可以通过Docker里面自带的`tlt-converter`工具来转换模型
`tlt-converter`工具将会将您训练出来的.etlt模型直接转换成可以部署在NVIDIA TensorRT和Deepstream 上的格式。对于x86设备，您可以直接复制docker中的工具到您自己的环境中
但是对于Jetson设备，您需要从下面的网址下载
[https://developer.nvidia.com/tlt-converter](https://developer.nvidia.com/tlt-converter)

In [ ]:
# 转换成TensorRT engine (FP32)
!tlt-converter -k $KEY \
               -d 3,300,300 \
               -o NMS \
               -e $USER_EXPERIMENT_DIR/export/trt--mobilenet_v2_epoch_040.engine \
               -m 1 \
               -t fp32 \
               -i nchw \
               $USER_EXPERIMENT_DIR/export/ssd_mobilenet_v2_epoch_040.etlt

# Uncomment to convert to TensorRT engine (INT8).
# !tlt-converter -k $KEY  \
#                -d 3,384,1248 \
#                -o NMS \
#                -c $USER_EXPERIMENT_DIR/export/cal.bin \
#                -e $USER_EXPERIMENT_DIR/export/trt.engine \
#                -b 8 \
#                -m 1 \
#                -t int8 \
#                -i nchw \
#                $USER_EXPERIMENT_DIR/export/ssd_resnet18_epoch_$EPOCH.etlt

In [ ]:
#查看转换好的推理引擎
print('Exported engine:')
print('------------')
!ls -lh $USER_EXPERIMENT_DIR/export/trt*.engine

## 10. 确认导出模型 <a class="anchor" id="head-10"></a>
通过下面的命令来确认导出的engine文件是否可以正确运行

In [ ]:
# 使用TensorRT engine文件进行推理
# 注意这里的tlt-infer 工具只支持batchsize为1，这点非常重要. 
# 所以请确认在使用tlt-converter工具时 ,使用`-m 1 --batch_size 1`为参数


!tlt-infer ssd --trt -p $USER_EXPERIMENT_DIR/export/trt--mobilenet_v2_epoch_040.engine \
                     -e $SPECS_DIR/ssd_train_mobilenet_v2_kitti.txt \
                     -i $DATA_DOWNLOAD_DIR/infer_images \
                     -o $USER_EXPERIMENT_DIR/ssd_infer_images \
                     -t 0.4

In [ ]:
!echo $KEY